## **Federated Learning for attack detection: 3 nodes sharing weights**
USES FILTERED DATASETS WITH PROPER PROPORTIONS

IDs from this file = **ID19xy** (x = 0 if experiment with dataset, x = 1 if epochs & iterations, y being integer equal or greater than 0)

In this file, experiments with different datasets, and number of epochs & iterations are done. The experiments are divided into sections, based on the elements being changed. 

### Static elements for all experiments (execute first)

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def preprocessing(data): 
    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = pd.DataFrame()

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Transform to One Hot Encoding the Categories - normal, attack
    data_labels.insert(0, 'normal', data['attack_cat'].replace('normal', 1).replace(['dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(1, 'attack', data['attack_cat'].replace(['dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 1).replace('normal', 0))

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

### Experiments with datasets

#### Dataset 3A

In [5]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3A-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/4261398225.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3A-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/4261398225.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/4261398225.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/4261398225.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or s

In [6]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [7]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1900.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [8]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [9]:
global_model = build_model((24,1,1))

In [13]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [14]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1900.hdf5')

Epoch 1/5
51/51 [==============================] - 9s 171ms/step - loss: 0.0159 - accuracy: 0.9952 - val_loss: 0.9235 - val_accuracy: 0.5644
Epoch 2/5
51/51 [==============================] - 8s 164ms/step - loss: 0.0151 - accuracy: 0.9956 - val_loss: 0.8882 - val_accuracy: 0.5842
Epoch 3/5
51/51 [==============================] - 8s 163ms/step - loss: 0.0148 - accuracy: 0.9958 - val_loss: 0.8259 - val_accuracy: 0.6175
Epoch 4/5
51/51 [==============================] - 8s 155ms/step - loss: 0.0144 - accuracy: 0.9957 - val_loss: 0.9242 - val_accuracy: 0.6018
Epoch 5/5
51/51 [==============================] - 8s 157ms/step - loss: 0.0143 - accuracy: 0.9959 - val_loss: 0.9261 - val_accuracy: 0.5938
Epoch 1/5
51/51 [==============================] - 9s 169ms/step - loss: 0.0138 - accuracy: 0.9959 - val_loss: 0.1039 - val_accuracy: 0.9557
Epoch 2/5
51/51 [==============================] - 8s 156ms/step - loss: 0.0132 - accuracy: 0.9959 - val_loss: 0.1672 - val_accuracy: 0.9275
Epoch 3/5
51/

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 0.6377 - accuracy: 0.7937
Epoch 1/5
51/51 [==============================] - 8s 156ms/step - loss: 0.0135 - accuracy: 0.9960 - val_loss: 1.4425 - val_accuracy: 0.5123
Epoch 2/5
51/51 [==============================] - 8s 150ms/step - loss: 0.0132 - accuracy: 0.9963 - val_loss: 0.9709 - val_accuracy: 0.6134
Epoch 3/5
51/51 [==============================] - 8s 163ms/step - loss: 0.0127 - accuracy: 0.9963 - val_loss: 1.2343 - val_accuracy: 0.5665
Epoch 4/5
51/51 [==============================] - 8s 161ms/step - loss: 0.0126 - accuracy: 0.9963 - val_loss: 0.9718 - val_accuracy: 0.6283
Epoch 5/5
51/51 [==============================] - 8s 152ms/step - loss: 0.0120 - accuracy: 0.9965 - val_loss: 1.0008 - val_accuracy: 0.6364
Epoch 1/5
51/51 [==============================] - 9s 173ms/step - loss: 0.0123 - accuracy: 0.9962 - val_loss: 0.1488 - val_accuracy: 0.9373
Epoch 2/5
51/51 [==============================] - 9s 180ms/ste

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 10s 6ms/step - loss: 0.9522 - accuracy: 0.6915
Epoch 1/5
51/51 [==============================] - 7s 141ms/step - loss: 0.0113 - accuracy: 0.9966 - val_loss: 0.8813 - val_accuracy: 0.6539
Epoch 2/5
51/51 [==============================] - 7s 141ms/step - loss: 0.0108 - accuracy: 0.9970 - val_loss: 1.2954 - val_accuracy: 0.5672
Epoch 3/5
51/51 [==============================] - 7s 139ms/step - loss: 0.0109 - accuracy: 0.9968 - val_loss: 1.4050 - val_accuracy: 0.5382
Epoch 4/5
51/51 [==============================] - 7s 138ms/step - loss: 0.0107 - accuracy: 0.9969 - val_loss: 1.3777 - val_accuracy: 0.5587
Epoch 5/5
51/51 [==============================] - 7s 140ms/step - loss: 0.0110 - accuracy: 0.9971 - val_loss: 1.1740 - val_accuracy: 0.6198
Epoch 1/5
51/51 [==============================] - 7s 144ms/step - loss: 0.0105 - accuracy: 0.9966 - val_loss: 0.0910 - val_accuracy: 0.9679
Epoch 2/5
51/51 [==============================] - 7s 143ms/st

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 1.4691 - accuracy: 0.5973


In [15]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.2747878432273865, 0.6377307772636414], [0.3942764103412628, 0.952224850654602], [0.7815492153167725, 1.4691256284713745]]
Accuracy for iterations:  [[0.9307458400726318, 0.793740451335907], [0.7619270086288452, 0.6915462017059326], [0.578266978263855, 0.5973079800605774]]
F1 for iterations:  [[0.9302972579774206, 0.7797400877371302], [0.7450686309044939, 0.644266187379188], [0.47160266863205885, 0.48299128383949]]
Precision for iterations:  [[0.9358871805179806, 0.8402202577709154], [0.8273035500292377, 0.7878036875045736], [0.7383901293036254, 0.7427696366275242]]
Recall for iterations:  [[0.9307458587747232, 0.7937404635617235], [0.76192702094715, 0.6915461745259028], [0.5782669666072628, 0.5973079997093657]]


#### Dataset 3B 

In [16]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3B-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/1530530892.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3B-Part2.csv')


In [17]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [18]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1901.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [19]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [20]:
global_model = build_model((24,1,1))

In [21]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [22]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1901.hdf5')

Epoch 1/5
51/51 [==============================] - 11s 219ms/step - loss: 0.1451 - accuracy: 0.9947 - val_loss: 2.6906e-05 - val_accuracy: 1.0000
Epoch 2/5
51/51 [==============================] - 11s 210ms/step - loss: 1.6899e-05 - accuracy: 1.0000 - val_loss: 1.4000e-05 - val_accuracy: 1.0000
Epoch 3/5
51/51 [==============================] - 11s 214ms/step - loss: 1.3473e-05 - accuracy: 1.0000 - val_loss: 1.2691e-05 - val_accuracy: 1.0000
Epoch 4/5
51/51 [==============================] - 11s 215ms/step - loss: 1.2182e-05 - accuracy: 1.0000 - val_loss: 1.1423e-05 - val_accuracy: 1.0000
Epoch 5/5
51/51 [==============================] - 11s 211ms/step - loss: 1.0923e-05 - accuracy: 1.0000 - val_loss: 1.0208e-05 - val_accuracy: 1.0000
Epoch 1/5
51/51 [==============================] - 11s 211ms/step - loss: 1.1348 - accuracy: 0.6250 - val_loss: 0.7937 - val_accuracy: 0.0000e+00
Epoch 2/5
51/51 [==============================] - 10s 206ms/step - loss: 0.3564 - accuracy: 0.8195 - val_lo

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 15s 8ms/step - loss: 0.3150 - accuracy: 0.9348
Epoch 1/5
51/51 [==============================] - 12s 228ms/step - loss: 0.0230 - accuracy: 0.9946 - val_loss: 1.7467e-05 - val_accuracy: 1.0000
Epoch 2/5
51/51 [==============================] - 10s 187ms/step - loss: 1.5220e-05 - accuracy: 1.0000 - val_loss: 1.4027e-05 - val_accuracy: 1.0000
Epoch 3/5
51/51 [==============================] - 12s 230ms/step - loss: 1.3226e-05 - accuracy: 1.0000 - val_loss: 1.2278e-05 - val_accuracy: 1.0000
Epoch 4/5
51/51 [==============================] - 11s 213ms/step - loss: 1.1300e-05 - accuracy: 1.0000 - val_loss: 1.0133e-05 - val_accuracy: 1.0000
Epoch 5/5
51/51 [==============================] - 8s 156ms/step - loss: 8.9546e-06 - accuracy: 1.0000 - val_loss: 7.6840e-06 - val_accuracy: 1.0000
Epoch 1/5
51/51 [==============================] - 8s 154ms/step - loss: 0.3452 - accuracy: 0.9229 - val_loss: 0.3366 - val_accuracy: 0.8853
Epoch 2/5
51/51 [=====

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 0.3254 - accuracy: 0.8530
Epoch 1/5
51/51 [==============================] - 7s 140ms/step - loss: 0.0365 - accuracy: 0.9842 - val_loss: 3.8351e-05 - val_accuracy: 1.0000
Epoch 2/5
51/51 [==============================] - 7s 143ms/step - loss: 3.2902e-05 - accuracy: 1.0000 - val_loss: 2.8228e-05 - val_accuracy: 1.0000
Epoch 3/5
51/51 [==============================] - 7s 142ms/step - loss: 2.6928e-05 - accuracy: 1.0000 - val_loss: 2.3732e-05 - val_accuracy: 1.0000
Epoch 4/5
51/51 [==============================] - 8s 154ms/step - loss: 2.2613e-05 - accuracy: 1.0000 - val_loss: 2.0042e-05 - val_accuracy: 1.0000
Epoch 5/5
51/51 [==============================] - 8s 154ms/step - loss: 1.9092e-05 - accuracy: 1.0000 - val_loss: 1.6951e-05 - val_accuracy: 1.0000
Epoch 1/5
51/51 [==============================] - 9s 170ms/step - loss: 0.1912 - accuracy: 0.9460 - val_loss: 0.2642 - val_accuracy: 0.8857
Epoch 2/5
51/51 [==========

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 10s 6ms/step - loss: 0.4162 - accuracy: 0.8060


In [23]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.2652433216571808, 0.3149508535861969], [0.2592286169528961, 0.32535701990127563], [0.36579328775405884, 0.41621941328048706]]
Accuracy for iterations:  [[0.9642554521560669, 0.9348070621490479], [0.8861929774284363, 0.8529571890830994], [0.8265359997749329, 0.80596524477005]]
F1 for iterations:  [[0.9642839717422446, 0.9350281469062873], [0.8854653116027342, 0.8524059562331278], [0.8230039430638397, 0.8032207517486496]]
Precision for iterations:  [[0.9653437421564772, 0.9386785368633961], [0.908275547087677, 0.8898185181049664], [0.8731852615179614, 0.8654173815944096]]
Recall for iterations:  [[0.9642554558999679, 0.9348070914771489], [0.8861929942446464, 0.8529572040979437], [0.8265359782640451, 0.8059652692000291]]


#### Dataset 3C

In [5]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3C-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_10828/769371098.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3C-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10828/769371098.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3C-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10828/769371098.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3C-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_10828/769371098.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or s

In [6]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [7]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1902.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [8]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [9]:
global_model = build_model((24,1,1))

In [10]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [11]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1902.hdf5')

Epoch 1/5
51/51 [==============================] - 9s 184ms/step - loss: 0.2717 - accuracy: 0.9496 - val_loss: 1.8632 - val_accuracy: 0.6191
Epoch 2/5
51/51 [==============================] - 8s 167ms/step - loss: 0.0333 - accuracy: 0.9920 - val_loss: 0.6609 - val_accuracy: 0.6334
Epoch 3/5
51/51 [==============================] - 9s 178ms/step - loss: 0.0265 - accuracy: 0.9920 - val_loss: 0.5897 - val_accuracy: 0.6451
Epoch 4/5
51/51 [==============================] - 10s 188ms/step - loss: 0.0245 - accuracy: 0.9921 - val_loss: 0.5972 - val_accuracy: 0.6299
Epoch 5/5
51/51 [==============================] - 9s 175ms/step - loss: 0.0229 - accuracy: 0.9926 - val_loss: 0.4975 - val_accuracy: 0.6545
Epoch 1/5
51/51 [==============================] - 9s 184ms/step - loss: 0.0201 - accuracy: 0.9932 - val_loss: 0.5549 - val_accuracy: 0.6656
Epoch 2/5
51/51 [==============================] - 9s 173ms/step - loss: 0.0174 - accuracy: 0.9942 - val_loss: 0.7060 - val_accuracy: 0.6777
Epoch 3/5
51

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 5s 3ms/step - loss: 0.2428 - accuracy: 0.9137
Epoch 1/5
51/51 [==============================] - 9s 176ms/step - loss: 0.0160 - accuracy: 0.9952 - val_loss: 0.4958 - val_accuracy: 0.7624
Epoch 2/5
51/51 [==============================] - 9s 177ms/step - loss: 0.0148 - accuracy: 0.9955 - val_loss: 0.4267 - val_accuracy: 0.7964
Epoch 3/5
51/51 [==============================] - 10s 187ms/step - loss: 0.0149 - accuracy: 0.9955 - val_loss: 0.3529 - val_accuracy: 0.8332
Epoch 4/5
51/51 [==============================] - 9s 169ms/step - loss: 0.0146 - accuracy: 0.9958 - val_loss: 0.4217 - val_accuracy: 0.8120
Epoch 5/5
51/51 [==============================] - 9s 167ms/step - loss: 0.0143 - accuracy: 0.9959 - val_loss: 0.5563 - val_accuracy: 0.7756
Epoch 1/5
51/51 [==============================] - 9s 171ms/step - loss: 0.0134 - accuracy: 0.9958 - val_loss: 0.5718 - val_accuracy: 0.7888
Epoch 2/5
51/51 [==============================] - 9s 181ms/st

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 4s 3ms/step - loss: 0.4918 - accuracy: 0.8303
Epoch 1/5
51/51 [==============================] - 9s 173ms/step - loss: 0.0136 - accuracy: 0.9959 - val_loss: 0.4103 - val_accuracy: 0.8341
Epoch 2/5
51/51 [==============================] - 9s 173ms/step - loss: 0.0134 - accuracy: 0.9960 - val_loss: 0.4997 - val_accuracy: 0.7970
Epoch 3/5
51/51 [==============================] - 9s 174ms/step - loss: 0.0130 - accuracy: 0.9962 - val_loss: 0.4139 - val_accuracy: 0.8321
Epoch 4/5
51/51 [==============================] - 9s 168ms/step - loss: 0.0128 - accuracy: 0.9963 - val_loss: 0.5008 - val_accuracy: 0.7989
Epoch 5/5
51/51 [==============================] - 9s 167ms/step - loss: 0.0125 - accuracy: 0.9962 - val_loss: 0.5066 - val_accuracy: 0.8039
Epoch 1/5
51/51 [==============================] - 9s 181ms/step - loss: 0.0114 - accuracy: 0.9962 - val_loss: 0.4088 - val_accuracy: 0.8370
Epoch 2/5
51/51 [==============================] - 9s 175ms/ste

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 5s 3ms/step - loss: 0.6750 - accuracy: 0.7741


In [12]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.09364178776741028, 0.24279256165027618], [0.15169090032577515, 0.49179762601852417], [0.17932547628879547, 0.6749835014343262]]
Accuracy for iterations:  [[0.9892416000366211, 0.913699746131897], [0.984494149684906, 0.8303058743476868], [0.9736918807029724, 0.7740681767463684]]
F1 for iterations:  [[0.989244454139426, 0.9125154658529658], [0.9844925725085262, 0.8223248973403091], [0.9736647138244197, 0.7557072757067063]]
Precision for iterations:  [[0.9892984207851672, 0.9206642174700433], [0.9844967255200144, 0.862110212068362], [0.9740386471388334, 0.8295610984638976]]
Recall for iterations:  [[0.9892415787782289, 0.9136997747584102], [0.9844941423938766, 0.8303058926106227], [0.9736918986824038, 0.7740681537455497]]


#### Dataset 3D

In [32]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3D-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3D-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3D-Part3.csv')

In [33]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [34]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1903.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [35]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [36]:
global_model = build_model((24,1,1))

In [37]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [38]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1903.hdf5')

Epoch 1/5
13/13 [==============================] - 2s 184ms/step - loss: 0.4675 - accuracy: 0.9527 - val_loss: 0.1457 - val_accuracy: 1.0000
Epoch 2/5
13/13 [==============================] - 2s 158ms/step - loss: 0.0398 - accuracy: 1.0000 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 3/5
13/13 [==============================] - 2s 148ms/step - loss: 4.1712e-04 - accuracy: 1.0000 - val_loss: 7.0437e-05 - val_accuracy: 1.0000
Epoch 4/5
13/13 [==============================] - 2s 147ms/step - loss: 4.2023e-05 - accuracy: 1.0000 - val_loss: 2.4089e-05 - val_accuracy: 1.0000
Epoch 5/5
13/13 [==============================] - 2s 172ms/step - loss: 2.0408e-05 - accuracy: 1.0000 - val_loss: 1.6970e-05 - val_accuracy: 1.0000
Epoch 1/5
13/13 [==============================] - 2s 174ms/step - loss: 5.5577 - accuracy: 0.0000e+00 - val_loss: 2.4075 - val_accuracy: 0.0000e+00
Epoch 2/5
13/13 [==============================] - 2s 172ms/step - loss: 1.4300 - accuracy: 0.0000e+00 - val_loss: 0.8229 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


4279/4279 [==============================] - 20s 5ms/step - loss: 0.9043 - accuracy: 0.4712


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1721/1721 [==============================] - 10s 6ms/step - loss: 0.9348 - accuracy: 0.4392


...

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


   1/1721 [..............................] - ETA: 0s - loss: 0.0314 - accuracy: 1.0000

c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1721/1721 [==============================] - 10s 6ms/step - loss: 2.0317 - accuracy: 0.4392


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.904284656047821, 0.9347654581069946], [1.915017008781433, 2.015558958053589], [1.931955099105835, 2.031741142272949]]
Accuracy for iterations:  [[0.4712086319923401, 0.4392392635345459], [0.4712086319923401, 0.4392392635345459], [0.4712086319923401, 0.4392392635345459]]
F1 for iterations:  [[0.30184375474744357, 0.2681015400046588], [0.30184375474744357, 0.2681015400046588], [0.30184375474744357, 0.2681015400046588]]
Precision for iterations:  [[0.2220375675826312, 0.19293113164997755], [0.2220375675826312, 0.19293113164997755], [0.2220375675826312, 0.19293113164997755]]
Recall for iterations:  [[0.4712086242659733, 0.43923926469519725], [0.4712086242659733, 0.43923926469519725], [0.4712086242659733, 0.43923926469519725]]


#### DATASET 3E

In [40]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3E-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3E-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3E-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/3505167362.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3E-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/3505167362.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3E-Part3.csv')


In [41]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [42]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1904.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [43]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [44]:
global_model = build_model((24,1,1))

In [45]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [46]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1904.hdf5')

Epoch 1/5
51/51 [==============================] - 11s 206ms/step - loss: 0.2494 - accuracy: 0.9741 - val_loss: 1.7371 - val_accuracy: 0.6682
Epoch 2/5
51/51 [==============================] - 10s 204ms/step - loss: 0.0321 - accuracy: 0.9922 - val_loss: 0.6133 - val_accuracy: 0.6769
Epoch 3/5
51/51 [==============================] - 10s 199ms/step - loss: 0.0259 - accuracy: 0.9923 - val_loss: 0.5528 - val_accuracy: 0.6831
Epoch 4/5
51/51 [==============================] - 10s 200ms/step - loss: 0.0240 - accuracy: 0.9925 - val_loss: 0.4401 - val_accuracy: 0.7158
Epoch 5/5
51/51 [==============================] - 10s 200ms/step - loss: 0.0224 - accuracy: 0.9928 - val_loss: 0.4545 - val_accuracy: 0.7054
Epoch 1/5
51/51 [==============================] - 11s 211ms/step - loss: 0.0207 - accuracy: 0.9929 - val_loss: 0.4590 - val_accuracy: 0.6928
Epoch 2/5
51/51 [==============================] - 10s 200ms/step - loss: 0.0175 - accuracy: 0.9943 - val_loss: 0.3321 - val_accuracy: 0.7971
Epoch 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 0.3110 - accuracy: 0.8855
Epoch 1/5
51/51 [==============================] - 11s 211ms/step - loss: 0.0158 - accuracy: 0.9953 - val_loss: 0.2856 - val_accuracy: 0.8544
Epoch 2/5
51/51 [==============================] - 10s 197ms/step - loss: 0.0148 - accuracy: 0.9958 - val_loss: 0.2502 - val_accuracy: 0.8787
Epoch 3/5
51/51 [==============================] - 10s 192ms/step - loss: 0.0148 - accuracy: 0.9957 - val_loss: 0.2082 - val_accuracy: 0.8970
Epoch 4/5
51/51 [==============================] - 10s 196ms/step - loss: 0.0143 - accuracy: 0.9957 - val_loss: 0.3660 - val_accuracy: 0.8272
Epoch 5/5
51/51 [==============================] - 10s 190ms/step - loss: 0.0141 - accuracy: 0.9961 - val_loss: 0.3437 - val_accuracy: 0.8410
Epoch 1/5
51/51 [==============================] - 11s 209ms/step - loss: 0.0140 - accuracy: 0.9956 - val_loss: 0.5234 - val_accuracy: 0.7766
Epoch 2/5
51/51 [==============================] - 10s 18

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 0.5411 - accuracy: 0.8061
Epoch 1/5
51/51 [==============================] - 15s 287ms/step - loss: 0.0137 - accuracy: 0.9962 - val_loss: 0.2725 - val_accuracy: 0.8724
Epoch 2/5
51/51 [==============================] - 16s 304ms/step - loss: 0.0131 - accuracy: 0.9962 - val_loss: 0.1963 - val_accuracy: 0.9073
Epoch 3/5
51/51 [==============================] - 16s 315ms/step - loss: 0.0129 - accuracy: 0.9961 - val_loss: 0.2689 - val_accuracy: 0.8810
Epoch 4/5
51/51 [==============================] - 14s 277ms/step - loss: 0.0123 - accuracy: 0.9964 - val_loss: 0.2363 - val_accuracy: 0.8952
Epoch 5/5
51/51 [==============================] - 14s 270ms/step - loss: 0.0132 - accuracy: 0.9965 - val_loss: 0.3690 - val_accuracy: 0.8366
Epoch 1/5
51/51 [==============================] - 16s 311ms/step - loss: 0.0121 - accuracy: 0.9960 - val_loss: 0.3706 - val_accuracy: 0.8272
Epoch 2/5
51/51 [==============================] - 14s 28

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 4ms/step - loss: 0.9030 - accuracy: 0.7022


In [47]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.15110553801059723, 0.3109899163246155], [0.19025111198425293, 0.5411234498023987], [0.32329997420310974, 0.9030158519744873]]
Accuracy for iterations:  [[0.9862178564071655, 0.8854537606239319], [0.9805647134780884, 0.8060560822486877], [0.8196850419044495, 0.702226996421814]]
F1 for iterations:  [[0.9862190979287047, 0.8828540652447012], [0.9805559171498381, 0.7943169729026466], [0.8122185104201383, 0.6598394951030476]]
Precision for iterations:  [[0.9862257108718581, 0.8989753447356438], [0.9806433972296336, 0.8475436656710653], [0.8601706484398761, 0.7931292593167546]]
Recall for iterations:  [[0.9862178269888107, 0.8854537528155199], [0.9805647258172895, 0.8060560924217104], [0.8196850623740103, 0.702226985395626]]


#### DATASET 3F

In [48]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/89903324.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/89903324.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/89903324.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part3.csv')


In [49]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [50]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1905.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [51]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [52]:
global_model = build_model((24,1,1))

In [53]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [54]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1905.hdf5')

Epoch 1/5
31/31 [==============================] - 8s 256ms/step - loss: 0.4924 - accuracy: 0.7761 - val_loss: 0.8399 - val_accuracy: 0.2471
Epoch 2/5
31/31 [==============================] - 8s 270ms/step - loss: 0.2105 - accuracy: 0.9173 - val_loss: 0.1920 - val_accuracy: 0.9600
Epoch 3/5
31/31 [==============================] - 9s 280ms/step - loss: 0.0963 - accuracy: 0.9758 - val_loss: 0.1099 - val_accuracy: 0.9891
Epoch 4/5
31/31 [==============================] - 8s 273ms/step - loss: 0.0644 - accuracy: 0.9840 - val_loss: 0.0896 - val_accuracy: 0.9961
Epoch 5/5
31/31 [==============================] - 9s 280ms/step - loss: 0.0500 - accuracy: 0.9873 - val_loss: 0.0789 - val_accuracy: 0.9977
Epoch 1/5
98/98 [==============================] - 28s 290ms/step - loss: 0.0212 - accuracy: 0.9931 - val_loss: 0.0183 - val_accuracy: 0.9924
Epoch 2/5
98/98 [==============================] - 27s 280ms/step - loss: 0.0159 - accuracy: 0.9948 - val_loss: 0.0145 - val_accuracy: 0.9938
Epoch 3/5
9

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 0.0820 - accuracy: 0.9797
Epoch 1/5
31/31 [==============================] - 8s 248ms/step - loss: 0.0399 - accuracy: 0.9884 - val_loss: 0.1251 - val_accuracy: 0.9981
Epoch 2/5
31/31 [==============================] - 8s 255ms/step - loss: 0.0357 - accuracy: 0.9885 - val_loss: 0.1696 - val_accuracy: 0.9914
Epoch 3/5
31/31 [==============================] - 8s 252ms/step - loss: 0.0346 - accuracy: 0.9886 - val_loss: 0.1173 - val_accuracy: 0.9976
Epoch 4/5
31/31 [==============================] - 8s 251ms/step - loss: 0.0339 - accuracy: 0.9887 - val_loss: 0.1719 - val_accuracy: 0.9871
Epoch 5/5
31/31 [==============================] - 8s 253ms/step - loss: 0.0330 - accuracy: 0.9889 - val_loss: 0.1321 - val_accuracy: 0.9971
Epoch 1/5
98/98 [==============================] - 26s 266ms/step - loss: 0.0161 - accuracy: 0.9953 - val_loss: 0.0187 - val_accuracy: 0.9930
Epoch 2/5
98/98 [==============================] - 26s 265ms/s

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 0.1226 - accuracy: 0.9650
Epoch 1/5
31/31 [==============================] - 8s 270ms/step - loss: 0.0345 - accuracy: 0.9885 - val_loss: 0.1601 - val_accuracy: 0.9917
Epoch 2/5
31/31 [==============================] - 7s 233ms/step - loss: 0.0316 - accuracy: 0.9891 - val_loss: 0.1942 - val_accuracy: 0.9533
Epoch 3/5
31/31 [==============================] - 8s 246ms/step - loss: 0.0304 - accuracy: 0.9894 - val_loss: 0.2536 - val_accuracy: 0.9004
Epoch 4/5
31/31 [==============================] - 8s 250ms/step - loss: 0.0302 - accuracy: 0.9897 - val_loss: 0.2048 - val_accuracy: 0.9328
Epoch 5/5
31/31 [==============================] - 7s 225ms/step - loss: 0.0295 - accuracy: 0.9897 - val_loss: 0.2492 - val_accuracy: 0.8519
Epoch 1/5
98/98 [==============================] - 24s 243ms/step - loss: 0.0141 - accuracy: 0.9959 - val_loss: 0.0119 - val_accuracy: 0.9955
Epoch 2/5
98/98 [==============================] - 24s 249ms/s

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 0.2586 - accuracy: 0.8924


In [55]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.036317676305770874, 0.08204517513513565], [0.030501162633299828, 0.12260642647743225], [0.03071054257452488, 0.25858744978904724]]
Accuracy for iterations:  [[0.99040287733078, 0.9796919226646423], [0.9919585585594177, 0.9650148749351501], [0.991337776184082, 0.8923744559288025]]
F1 for iterations:  [[0.9904071741676135, 0.979692834827217], [0.99196169120534, 0.9649327039299128], [0.9913401921767304, 0.8901177736525885]]
Precision for iterations:  [[0.9905810819099359, 0.9796940042926382], [0.9920846030546682, 0.9653995430314516], [0.9914000457911404, 0.9047318564404355]]
Recall for iterations:  [[0.9904028747553244, 0.9796919276320569], [0.9919585731397353, 0.9650148950083557], [0.9913377545356277, 0.8923744823076364]]


#### DATASET 3G

In [56]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3G-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3G-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3G-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/1702362103.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3G-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/1702362103.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3G-Part3.csv')


In [57]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [58]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1906.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [59]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [60]:
global_model = build_model((24,1,1))

In [61]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [62]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1906.hdf5')

Epoch 1/5
20/20 [==============================] - 5s 226ms/step - loss: 0.4891 - accuracy: 0.9319 - val_loss: 1.7899 - val_accuracy: 0.1462
Epoch 2/5
20/20 [==============================] - 4s 224ms/step - loss: 0.1631 - accuracy: 0.9574 - val_loss: 0.9596 - val_accuracy: 0.3860
Epoch 3/5
20/20 [==============================] - 4s 221ms/step - loss: 0.0727 - accuracy: 0.9775 - val_loss: 0.6613 - val_accuracy: 0.6902
Epoch 4/5
20/20 [==============================] - 5s 236ms/step - loss: 0.0543 - accuracy: 0.9801 - val_loss: 0.4259 - val_accuracy: 0.8409
Epoch 5/5
20/20 [==============================] - 4s 222ms/step - loss: 0.0480 - accuracy: 0.9836 - val_loss: 0.4433 - val_accuracy: 0.8273
Epoch 1/5
54/54 [==============================] - 15s 269ms/step - loss: 0.0237 - accuracy: 0.9918 - val_loss: 0.4141 - val_accuracy: 0.6818
Epoch 2/5
54/54 [==============================] - 14s 268ms/step - loss: 0.0196 - accuracy: 0.9932 - val_loss: 0.3257 - val_accuracy: 0.7786
Epoch 3/5
5

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 0.1458 - accuracy: 0.9425
Epoch 1/5
20/20 [==============================] - 6s 283ms/step - loss: 0.0281 - accuracy: 0.9889 - val_loss: 0.2057 - val_accuracy: 0.9557
Epoch 2/5
20/20 [==============================] - 5s 254ms/step - loss: 0.0261 - accuracy: 0.9901 - val_loss: 0.2162 - val_accuracy: 0.9416
Epoch 3/5
20/20 [==============================] - 5s 251ms/step - loss: 0.0257 - accuracy: 0.9906 - val_loss: 0.2761 - val_accuracy: 0.8898
Epoch 4/5
20/20 [==============================] - 5s 259ms/step - loss: 0.0248 - accuracy: 0.9908 - val_loss: 0.2941 - val_accuracy: 0.8672
Epoch 5/5
20/20 [==============================] - 5s 266ms/step - loss: 0.0249 - accuracy: 0.9911 - val_loss: 0.3108 - val_accuracy: 0.8454
Epoch 1/5
54/54 [==============================] - 16s 289ms/step - loss: 0.0152 - accuracy: 0.9954 - val_loss: 0.3007 - val_accuracy: 0.8444
Epoch 2/5
54/54 [==============================] - 16s 290ms/s

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 13s 7ms/step - loss: 0.3445 - accuracy: 0.8589
Epoch 1/5
20/20 [==============================] - 7s 374ms/step - loss: 0.0241 - accuracy: 0.9905 - val_loss: 0.2944 - val_accuracy: 0.8441
Epoch 2/5
20/20 [==============================] - 7s 335ms/step - loss: 0.0218 - accuracy: 0.9918 - val_loss: 0.2871 - val_accuracy: 0.8373
Epoch 3/5
20/20 [==============================] - 6s 305ms/step - loss: 0.0217 - accuracy: 0.9920 - val_loss: 0.3633 - val_accuracy: 0.7793
Epoch 4/5
20/20 [==============================] - 7s 368ms/step - loss: 0.0212 - accuracy: 0.9922 - val_loss: 0.2987 - val_accuracy: 0.8327
Epoch 5/5
20/20 [==============================] - 7s 343ms/step - loss: 0.0210 - accuracy: 0.9920 - val_loss: 0.4262 - val_accuracy: 0.7497
Epoch 1/5
54/54 [==============================] - 20s 374ms/step - loss: 0.0134 - accuracy: 0.9962 - val_loss: 0.3780 - val_accuracy: 0.8347
Epoch 2/5
54/54 [==============================] - 21s 395ms/

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 0.6137 - accuracy: 0.7549


In [63]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.08730008453130722, 0.1458355337381363], [0.10025162994861603, 0.34445902705192566], [0.17731809616088867, 0.6136548519134521]]
Accuracy for iterations:  [[0.9883286356925964, 0.9425270557403564], [0.9878538846969604, 0.8588970303535461], [0.9757442474365234, 0.754886269569397]]
F1 for iterations:  [[0.9883332961354578, 0.9421687176171072], [0.9878556222806287, 0.8541542106213706], [0.975722963697017, 0.7314377050846614]]
Precision for iterations:  [[0.9884676474774722, 0.9446438318790938], [0.987870997511829, 0.8807548339812601], [0.9760070100088785, 0.8195889618985991]]
Recall for iterations:  [[0.9883286102427766, 0.942527065320061], [0.9878538666043414, 0.8588970427959021], [0.9757442519501008, 0.754886289326455]]


### Weighted mean use for unbalanced partitions 

#### Dataset 3F

In [64]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/89903324.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/89903324.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/89903324.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3F-Part3.csv')


In [65]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [66]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1907.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [71]:
def aggregate_weighted(w_list, sizes):
    total_size = np.sum(sizes)
    weighted_avg_w = []
    for layer_weights in zip(*w_list):
        weighted_layer = np.average(layer_weights, axis=0, weights=[size / total_size for size in sizes])
        weighted_avg_w.append(weighted_layer)
    return weighted_avg_w


In [68]:
global_model = build_model((24,1,1))

In [69]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [72]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    sizes = []  # List to store the sizes of local training datasets
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())
        sizes.append(len(x))  # Append the size of the local dataset

    avg_w = aggregate_weighted(w_list, sizes)
    global_model.set_weights(avg_w) # apply weighted average weights to global model

    # Evaluation
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1907.hdf5')

Epoch 1/5
31/31 [==============================] - 6s 186ms/step - loss: 0.0501 - accuracy: 0.9857 - val_loss: 0.1056 - val_accuracy: 0.9977
Epoch 2/5
31/31 [==============================] - 6s 198ms/step - loss: 0.0374 - accuracy: 0.9886 - val_loss: 0.1269 - val_accuracy: 0.9980
Epoch 3/5
31/31 [==============================] - 6s 200ms/step - loss: 0.0353 - accuracy: 0.9886 - val_loss: 0.1201 - val_accuracy: 0.9981
Epoch 4/5
31/31 [==============================] - 6s 201ms/step - loss: 0.0343 - accuracy: 0.9887 - val_loss: 0.1037 - val_accuracy: 0.9988
Epoch 5/5
31/31 [==============================] - 6s 191ms/step - loss: 0.0335 - accuracy: 0.9888 - val_loss: 0.1436 - val_accuracy: 0.9938
Epoch 1/5
98/98 [==============================] - 19s 191ms/step - loss: 0.0154 - accuracy: 0.9955 - val_loss: 0.0136 - val_accuracy: 0.9944
Epoch 2/5
98/98 [==============================] - 18s 188ms/step - loss: 0.0131 - accuracy: 0.9963 - val_loss: 0.0120 - val_accuracy: 0.9954
Epoch 3/5
9

In [73]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.041488200426101685, 0.30425289273262024], [0.04384875297546387, 0.44927117228507996], [0.055174168199300766, 0.6336178183555603]]
Accuracy for iterations:  [[0.9816603064537048, 0.8555910587310791], [0.9829384684562683, 0.8191527724266052], [0.9784393310546875, 0.7773014307022095]]
F1 for iterations:  [[0.9816510419603829, 0.8504546101438504], [0.9829314134438815, 0.8093805509054842], [0.978420790450338, 0.7593355210117334]]
Precision for iterations:  [[0.9817585263491, 0.8790597718739682], [0.9830056154340061, 0.8564019481134787], [0.9786980975986554, 0.8332100998030972]]
Recall for iterations:  [[0.9816602880598323, 0.855591077526702], [0.9829384440094656, 0.8191528009881567], [0.9784393350667563, 0.7773014604374047]]


#### Dataset Filt3G

In [74]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3G-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3G-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3G-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/1702362103.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3G-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_5216/1702362103.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt3G-Part3.csv')


In [75]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [76]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1908.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [77]:
def aggregate_weighted(w_list, sizes):
    total_size = np.sum(sizes)
    weighted_avg_w = []
    for layer_weights in zip(*w_list):
        weighted_layer = np.average(layer_weights, axis=0, weights=[size / total_size for size in sizes])
        weighted_avg_w.append(weighted_layer)
    return weighted_avg_w


In [78]:
global_model = build_model((24,1,1))

In [79]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [82]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    sizes = []  # List to store the sizes of local training datasets
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())
        sizes.append(len(x))  # Append the size of the local dataset

    avg_w = aggregate_weighted(w_list, sizes)
    global_model.set_weights(avg_w) # apply weighted average weights to global model

    # Evaluation
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1908.hdf5')

Epoch 1/5
20/20 [==============================] - 4s 183ms/step - loss: 0.0209 - accuracy: 0.9919 - val_loss: 0.3879 - val_accuracy: 0.7804
Epoch 2/5
20/20 [==============================] - 4s 202ms/step - loss: 0.0181 - accuracy: 0.9924 - val_loss: 0.4362 - val_accuracy: 0.7700
Epoch 3/5
20/20 [==============================] - 3s 165ms/step - loss: 0.0176 - accuracy: 0.9926 - val_loss: 0.4383 - val_accuracy: 0.7699
Epoch 4/5
20/20 [==============================] - 3s 169ms/step - loss: 0.0174 - accuracy: 0.9926 - val_loss: 0.4256 - val_accuracy: 0.7785
Epoch 5/5
20/20 [==============================] - 3s 175ms/step - loss: 0.0171 - accuracy: 0.9926 - val_loss: 0.4988 - val_accuracy: 0.7559
Epoch 1/5
54/54 [==============================] - 11s 204ms/step - loss: 0.0112 - accuracy: 0.9966 - val_loss: 0.4942 - val_accuracy: 0.8091
Epoch 2/5
54/54 [==============================] - 10s 184ms/step - loss: 0.0102 - accuracy: 0.9970 - val_loss: 0.4011 - val_accuracy: 0.8404
Epoch 3/5
5

In [83]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.2012778520584106, 1.7955865859985352], [1.588597297668457, 2.1838297843933105], [2.335428237915039, 2.7038397789001465]]
Accuracy for iterations:  [[0.5859286189079285, 0.5935842394828796], [0.5856875777244568, 0.5906415581703186], [0.5812323093414307, 0.5796701312065125]]
F1 for iterations:  [[0.48695020766989316, 0.4751437524371377], [0.4864708979735109, 0.46886849799094027], [0.4775546323239458, 0.44561481820703497]]
Precision for iterations:  [[0.7373255775396503, 0.7432347534352273], [0.7373619705289497, 0.743821323155352], [0.7380911842209517, 0.7398820221090562]]
Recall for iterations:  [[0.5859285985567794, 0.5935842476204316], [0.5856875748634199, 0.5906415752379568], [0.5812322884104122, 0.5796701300588535]]


### Experimenting with epochs and iterations 

Use of dataset 3C, considering it contains an equal partition of the data between the 3 nodes. 

#### #Epochs = 10, #Iterations = 10

In [72]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low

In [73]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 10

In [74]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1910.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [75]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [76]:
global_model = build_model((24,1,1))

In [77]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [78]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1910.hdf5')

Epoch 1/10
54/54 [==============================] - 11s 199ms/step - loss: 0.2544 - accuracy: 0.9327 - val_loss: 1.8450 - val_accuracy: 0.6399
Epoch 2/10
54/54 [==============================] - 11s 197ms/step - loss: 0.0321 - accuracy: 0.9925 - val_loss: 0.7600 - val_accuracy: 0.6422
Epoch 3/10
54/54 [==============================] - 10s 192ms/step - loss: 0.0254 - accuracy: 0.9926 - val_loss: 0.5458 - val_accuracy: 0.6635
Epoch 4/10
54/54 [==============================] - 11s 196ms/step - loss: 0.0237 - accuracy: 0.9928 - val_loss: 0.4793 - val_accuracy: 0.6839
Epoch 5/10
54/54 [==============================] - 10s 194ms/step - loss: 0.0220 - accuracy: 0.9930 - val_loss: 0.5456 - val_accuracy: 0.6567
Epoch 6/10
54/54 [==============================] - 10s 193ms/step - loss: 0.0206 - accuracy: 0.9934 - val_loss: 0.4470 - val_accuracy: 0.7066
Epoch 7/10
54/54 [==============================] - 10s 192ms/step - loss: 0.0188 - accuracy: 0.9942 - val_loss: 0.5359 - val_accuracy: 0.6935

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 0.3768 - accuracy: 0.8759
Epoch 1/10
54/54 [==============================] - 11s 200ms/step - loss: 0.0137 - accuracy: 0.9961 - val_loss: 0.4771 - val_accuracy: 0.8012
Epoch 2/10
54/54 [==============================] - 11s 207ms/step - loss: 0.0130 - accuracy: 0.9960 - val_loss: 0.4833 - val_accuracy: 0.8056
Epoch 3/10
54/54 [==============================] - 11s 199ms/step - loss: 0.0127 - accuracy: 0.9963 - val_loss: 0.6188 - val_accuracy: 0.7750
Epoch 4/10
54/54 [==============================] - 11s 198ms/step - loss: 0.0127 - accuracy: 0.9963 - val_loss: 0.3453 - val_accuracy: 0.8471
Epoch 5/10
54/54 [==============================] - 11s 200ms/step - loss: 0.0125 - accuracy: 0.9961 - val_loss: 0.6800 - val_accuracy: 0.7706
Epoch 6/10
54/54 [==============================] - 11s 198ms/step - loss: 0.0122 - accuracy: 0.9965 - val_loss: 0.4442 - val_accuracy: 0.8306
Epoch 7/10
54/54 [==============================] -

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 10s 6ms/step - loss: 1.0575 - accuracy: 0.6811 0s - loss: 1.0662 - accuracy: 0.
Epoch 1/10
54/54 [==============================] - 11s 196ms/step - loss: 0.0096 - accuracy: 0.9973 - val_loss: 0.5719 - val_accuracy: 0.8128
Epoch 2/10
54/54 [==============================] - 10s 193ms/step - loss: 0.0093 - accuracy: 0.9973 - val_loss: 0.5635 - val_accuracy: 0.8080
Epoch 3/10
54/54 [==============================] - 10s 191ms/step - loss: 0.0087 - accuracy: 0.9974 - val_loss: 0.6817 - val_accuracy: 0.7872
Epoch 4/10
54/54 [==============================] - 10s 188ms/step - loss: 0.0091 - accuracy: 0.9974 - val_loss: 0.3999 - val_accuracy: 0.8470
Epoch 5/10
54/54 [==============================] - 10s 191ms/step - loss: 0.0089 - accuracy: 0.9972 - val_loss: 0.5172 - val_accuracy: 0.8254
Epoch 6/10
54/54 [==============================] - 10s 190ms/step - loss: 0.0089 - accuracy: 0.9975 - val_loss: 0.5384 - val_accuracy: 0.8198
Epoch 7/10
54/54 

...

1721/1721 [==============================] - 8s 5ms/step - loss: 3.6558 - accuracy: 0.6425
Epoch 1/10
54/54 [==============================] - 11s 198ms/step - loss: 0.0056 - accuracy: 0.9985 - val_loss: 0.7006 - val_accuracy: 0.7890
Epoch 2/10
54/54 [==============================] - 10s 193ms/step - loss: 0.0052 - accuracy: 0.9984 - val_loss: 0.6449 - val_accuracy: 0.7970
Epoch 3/10
54/54 [==============================] - 10s 193ms/step - loss: 0.0052 - accuracy: 0.9984 - val_loss: 0.3634 - val_accuracy: 0.8604
Epoch 4/10
54/54 [==============================] - 10s 192ms/step - loss: 0.0054 - accuracy: 0.9984 - val_loss: 0.4947 - val_accuracy: 0.8304
Epoch 5/10
54/54 [==============================] - 10s 191ms/step - loss: 0.0049 - accuracy: 0.9985 - val_loss: 0.6775 - val_accuracy: 0.8014
Epoch 6/10
54/54 [==============================] - 10s 189ms/step - loss: 0.0051 - accuracy: 0.9986 - val_loss: 0.5711 - val_accuracy: 0.8132
Epoch 7/10
54/54 [==============================] -

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 4.0920 - accuracy: 0.6729
Epoch 1/10
54/54 [==============================] - 11s 197ms/step - loss: 0.0047 - accuracy: 0.9985 - val_loss: 0.6507 - val_accuracy: 0.8058
Epoch 2/10
54/54 [==============================] - 11s 195ms/step - loss: 0.0046 - accuracy: 0.9986 - val_loss: 0.6399 - val_accuracy: 0.8050
Epoch 3/10
54/54 [==============================] - 11s 196ms/step - loss: 0.0045 - accuracy: 0.9985 - val_loss: 0.6329 - val_accuracy: 0.8100
Epoch 4/10
54/54 [==============================] - 11s 199ms/step - loss: 0.0045 - accuracy: 0.9986 - val_loss: 0.5471 - val_accuracy: 0.8214
Epoch 5/10
54/54 [==============================] - 10s 193ms/step - loss: 0.0049 - accuracy: 0.9983 - val_loss: 0.5311 - val_accuracy: 0.8320
Epoch 6/10
54/54 [==============================] - 10s 194ms/step - loss: 0.0044 - accuracy: 0.9986 - val_loss: 0.7088 - val_accuracy: 0.7913
Epoch 7/10
54/54 [==============================] -

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 4.2869 - accuracy: 0.7279


In [79]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.1511470228433609, 0.37683457136154175], [0.3587162494659424, 1.0574772357940674], [0.8660457134246826, 2.043614387512207], [1.6211916208267212, 2.6672563552856445], [2.25840163230896, 2.9753661155700684], [3.4451019763946533, 3.571256399154663], [3.8306243419647217, 3.4602181911468506], [4.477360248565674, 3.655755043029785], [5.541928291320801, 4.091959476470947], [6.338571071624756, 4.286871910095215]]
Accuracy for iterations:  [[0.9788848757743835, 0.8759173154830933], [0.7929971814155579, 0.6811015009880066], [0.5881343483924866, 0.5786892175674438], [0.5880978107452393, 0.5743842124938965], [0.5904861092567444, 0.5772542357444763], [0.5903181433677673, 0.5788345336914062], [0.5935537219047546, 0.6072622537612915], [0.5936924815177917, 0.6425379514694214], [0.5941160917282104, 0.6728910803794861], [0.593298077583313, 0.7279117703437805]]
F1 for iterations:  [[0.9788777632026339, 0.872614195868444], [0.7821209882007997, 0.6280498098219511], [0.4910611551908

#### #Epochs = 5, #Iterations = 5

In [5]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16700/1685572531.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16700/1685572531.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16700/1685572531.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16700/1685572531.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low

In [6]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [7]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1911.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [8]:
def aggregate(w_list): 
    avg_w = np.mean(w_list, axis=0)
    return avg_w

In [9]:
global_model = build_model((24,1,1))

In [10]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [11]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1911.hdf5')

Epoch 1/5
54/54 [==============================] - 12s 223ms/step - loss: 0.2645 - accuracy: 0.9722 - val_loss: 1.9327 - val_accuracy: 0.6399
Epoch 2/5
54/54 [==============================] - 12s 230ms/step - loss: 0.0342 - accuracy: 0.9921 - val_loss: 0.7014 - val_accuracy: 0.6454
Epoch 3/5
54/54 [==============================] - 13s 238ms/step - loss: 0.0256 - accuracy: 0.9925 - val_loss: 0.4979 - val_accuracy: 0.6941
Epoch 4/5
54/54 [==============================] - 12s 225ms/step - loss: 0.0239 - accuracy: 0.9925 - val_loss: 0.4756 - val_accuracy: 0.6957
Epoch 5/5
54/54 [==============================] - 12s 219ms/step - loss: 0.0224 - accuracy: 0.9927 - val_loss: 0.4330 - val_accuracy: 0.7127
Epoch 1/5
54/54 [==============================] - 12s 226ms/step - loss: 0.0200 - accuracy: 0.9934 - val_loss: 0.4135 - val_accuracy: 0.7392
Epoch 2/5
54/54 [==============================] - 12s 226ms/step - loss: 0.0171 - accuracy: 0.9946 - val_loss: 0.3682 - val_accuracy: 0.7957
Epoch 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 6s 4ms/step - loss: 0.2767 - accuracy: 0.8953
Epoch 1/5
54/54 [==============================] - 12s 220ms/step - loss: 0.0151 - accuracy: 0.9957 - val_loss: 0.3714 - val_accuracy: 0.8160
Epoch 2/5
54/54 [==============================] - 12s 217ms/step - loss: 0.0145 - accuracy: 0.9959 - val_loss: 0.4096 - val_accuracy: 0.8142
Epoch 3/5
54/54 [==============================] - 12s 221ms/step - loss: 0.0147 - accuracy: 0.9958 - val_loss: 0.4040 - val_accuracy: 0.8163
Epoch 4/5
54/54 [==============================] - 13s 238ms/step - loss: 0.0140 - accuracy: 0.9960 - val_loss: 0.4228 - val_accuracy: 0.8181
Epoch 5/5
54/54 [==============================] - 12s 231ms/step - loss: 0.0138 - accuracy: 0.9963 - val_loss: 0.3975 - val_accuracy: 0.8190
Epoch 1/5
54/54 [==============================] - 12s 219ms/step - loss: 0.0138 - accuracy: 0.9960 - val_loss: 0.5510 - val_accuracy: 0.7894
Epoch 2/5
54/54 [==============================] - 12s 22

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 0.6133 - accuracy: 0.7966
Epoch 1/5
54/54 [==============================] - 12s 228ms/step - loss: 0.0135 - accuracy: 0.9965 - val_loss: 0.4723 - val_accuracy: 0.8211
Epoch 2/5
54/54 [==============================] - 13s 234ms/step - loss: 0.0127 - accuracy: 0.9966 - val_loss: 0.4651 - val_accuracy: 0.8210
Epoch 3/5
54/54 [==============================] - 12s 225ms/step - loss: 0.0123 - accuracy: 0.9967 - val_loss: 0.6406 - val_accuracy: 0.7696
Epoch 4/5
54/54 [==============================] - 12s 219ms/step - loss: 0.0121 - accuracy: 0.9966 - val_loss: 0.5718 - val_accuracy: 0.7938
Epoch 5/5
54/54 [==============================] - 12s 226ms/step - loss: 0.0121 - accuracy: 0.9966 - val_loss: 0.5102 - val_accuracy: 0.8151
Epoch 1/5
54/54 [==============================] - 12s 228ms/step - loss: 0.0113 - accuracy: 0.9965 - val_loss: 0.4983 - val_accuracy: 0.8257
Epoch 2/5
54/54 [==============================] - 12s 22

...

1721/1721 [==============================] - 7s 4ms/step - loss: 0.8982 - accuracy: 0.7184
Epoch 1/5
54/54 [==============================] - 13s 235ms/step - loss: 0.0109 - accuracy: 0.9969 - val_loss: 0.3934 - val_accuracy: 0.8504
Epoch 2/5
54/54 [==============================] - 13s 240ms/step - loss: 0.0106 - accuracy: 0.9969 - val_loss: 0.7582 - val_accuracy: 0.7619
Epoch 3/5
54/54 [==============================] - 14s 252ms/step - loss: 0.0102 - accuracy: 0.9971 - val_loss: 0.6039 - val_accuracy: 0.8006
Epoch 4/5
54/54 [==============================] - 12s 228ms/step - loss: 0.0099 - accuracy: 0.9973 - val_loss: 0.3868 - val_accuracy: 0.8522
Epoch 5/5
54/54 [==============================] - 12s 221ms/step - loss: 0.0097 - accuracy: 0.9971 - val_loss: 0.5528 - val_accuracy: 0.8157
Epoch 1/5
54/54 [==============================] - 12s 224ms/step - loss: 0.0094 - accuracy: 0.9969 - val_loss: 0.6403 - val_accuracy: 0.8069
Epoch 2/5
54/54 [==============================] - 12s 22

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 6s 4ms/step - loss: 1.1148 - accuracy: 0.6839
Epoch 1/5
54/54 [==============================] - 12s 227ms/step - loss: 0.0093 - accuracy: 0.9974 - val_loss: 0.6650 - val_accuracy: 0.8055
Epoch 2/5
54/54 [==============================] - 12s 220ms/step - loss: 0.0095 - accuracy: 0.9976 - val_loss: 0.5259 - val_accuracy: 0.8258
Epoch 3/5
54/54 [==============================] - 12s 218ms/step - loss: 0.0084 - accuracy: 0.9977 - val_loss: 0.7564 - val_accuracy: 0.7706
Epoch 4/5
54/54 [==============================] - 12s 220ms/step - loss: 0.0081 - accuracy: 0.9978 - val_loss: 0.6562 - val_accuracy: 0.8006
Epoch 5/5
54/54 [==============================] - 12s 222ms/step - loss: 0.0084 - accuracy: 0.9977 - val_loss: 0.8334 - val_accuracy: 0.7509
Epoch 1/5
54/54 [==============================] - 15s 277ms/step - loss: 0.0083 - accuracy: 0.9973 - val_loss: 0.5401 - val_accuracy: 0.8260
Epoch 2/5
54/54 [==============================] - 12s 22

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 6s 4ms/step - loss: 1.2401 - accuracy: 0.6627


In [12]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.11703264713287354, 0.2767493426799774], [0.19591358304023743, 0.6133018732070923], [0.1857309639453888, 0.898158848285675], [0.18993905186653137, 1.1148115396499634], [0.2746482193470001, 1.2400809526443481]]
Accuracy for iterations:  [[0.9879488348960876, 0.8953171372413635], [0.9742615818977356, 0.7965922951698303], [0.9685500860214233, 0.7183572053909302], [0.9675859808921814, 0.6838988661766052], [0.8596073389053345, 0.6627007126808167]]
F1 for iterations:  [[0.9879512311722966, 0.893262750243958], [0.9742392353914436, 0.7831071675087792], [0.9684930401739139, 0.6826180191237142], [0.9675242307820903, 0.6323115245188365], [0.8560237382588983, 0.5989273184549058]]
Precision for iterations:  [[0.9879809146711573, 0.9066074075223175], [0.9745164002796086, 0.842080209017125], [0.9694262389720655, 0.8012362684054279], [0.9685306758294475, 0.786376489534689], [0.8846263110674074, 0.7781947755840517]]
Recall for iterations:  [[0.9879488153320284, 0.89531715469011

#### #Epochs = 5, #Iterations = 10

In [80]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10
# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [81]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1912.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [82]:
global_model = build_model((24,1,1))

In [83]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1912.hdf5')

Epoch 1/5


54/54 [==============================] - 11s 199ms/step - loss: 0.2511 - accuracy: 0.9069 - val_loss: 1.9361 - val_accuracy: 0.6399
Epoch 2/5
54/54 [==============================] - 11s 195ms/step - loss: 0.0319 - accuracy: 0.9925 - val_loss: 0.6612 - val_accuracy: 0.6454
Epoch 3/5
54/54 [==============================] - 10s 194ms/step - loss: 0.0251 - accuracy: 0.9926 - val_loss: 0.6359 - val_accuracy: 0.6476
Epoch 4/5
54/54 [==============================] - 10s 193ms/step - loss: 0.0235 - accuracy: 0.9927 - val_loss: 0.4588 - val_accuracy: 0.6863
Epoch 5/5
54/54 [==============================] - 10s 190ms/step - loss: 0.0220 - accuracy: 0.9928 - val_loss: 0.6081 - val_accuracy: 0.6489
Epoch 1/5
54/54 [==============================] - 10s 193ms/step - loss: 0.0200 - accuracy: 0.9934 - val_loss: 0.4392 - val_accuracy: 0.7208
Epoch 2/5
54/54 [==============================] - 11s 196ms/step - loss: 0.0170 - accuracy: 0.9945 - val_loss: 0.4354 - val_accuracy: 0.7638
Epoch 3/5
54/54 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 0.2919 - accuracy: 0.8903
Epoch 1/5
54/54 [==============================] - 10s 183ms/step - loss: 0.0151 - accuracy: 0.9955 - val_loss: 0.3542 - val_accuracy: 0.8211
Epoch 2/5
54/54 [==============================] - 10s 179ms/step - loss: 0.0146 - accuracy: 0.9958 - val_loss: 0.4645 - val_accuracy: 0.7932
Epoch 3/5
54/54 [==============================] - 9s 174ms/step - loss: 0.0142 - accuracy: 0.9959 - val_loss: 0.5226 - val_accuracy: 0.7890
Epoch 4/5
54/54 [==============================] - 10s 177ms/step - loss: 0.0140 - accuracy: 0.9961 - val_loss: 0.4800 - val_accuracy: 0.7964
Epoch 5/5
54/54 [==============================] - 10s 178ms/step - loss: 0.0140 - accuracy: 0.9960 - val_loss: 0.4435 - val_accuracy: 0.8102
Epoch 1/5
54/54 [==============================] - 10s 179ms/step - loss: 0.0133 - accuracy: 0.9960 - val_loss: 0.4236 - val_accuracy: 0.8256
Epoch 2/5
54/54 [==============================] - 10s 178

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


...

1721/1721 [==============================] - 9s 5ms/step - loss: 2.8040 - accuracy: 0.5842
Epoch 1/5
54/54 [==============================] - 10s 187ms/step - loss: 0.0073 - accuracy: 0.9981 - val_loss: 0.8659 - val_accuracy: 0.7804
Epoch 2/5
54/54 [==============================] - 10s 179ms/step - loss: 0.0068 - accuracy: 0.9981 - val_loss: 0.6685 - val_accuracy: 0.7993
Epoch 3/5
54/54 [==============================] - 10s 181ms/step - loss: 0.0069 - accuracy: 0.9981 - val_loss: 0.6449 - val_accuracy: 0.8058
Epoch 4/5
54/54 [==============================] - 10s 183ms/step - loss: 0.0070 - accuracy: 0.9982 - val_loss: 0.6463 - val_accuracy: 0.8061
Epoch 5/5
54/54 [==============================] - 10s 177ms/step - loss: 0.0066 - accuracy: 0.9983 - val_loss: 0.5415 - val_accuracy: 0.8284
Epoch 1/5
54/54 [==============================] - 10s 181ms/step - loss: 0.0070 - accuracy: 0.9978 - val_loss: 0.5380 - val_accuracy: 0.8321
Epoch 2/5
54/54 [==============================] - 10s 17

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 2.7545 - accuracy: 0.5972
Epoch 1/5
54/54 [==============================] - 11s 205ms/step - loss: 0.0067 - accuracy: 0.9982 - val_loss: 0.7035 - val_accuracy: 0.8026
Epoch 2/5
54/54 [==============================] - 10s 191ms/step - loss: 0.0070 - accuracy: 0.9980 - val_loss: 0.7450 - val_accuracy: 0.7860
Epoch 3/5
54/54 [==============================] - 10s 182ms/step - loss: 0.0067 - accuracy: 0.9982 - val_loss: 0.6474 - val_accuracy: 0.8085
Epoch 4/5
54/54 [==============================] - 10s 183ms/step - loss: 0.0065 - accuracy: 0.9983 - val_loss: 0.7878 - val_accuracy: 0.7786
Epoch 5/5
54/54 [==============================] - 10s 181ms/step - loss: 0.0064 - accuracy: 0.9983 - val_loss: 0.9783 - val_accuracy: 0.7380
Epoch 1/5
54/54 [==============================] - 10s 181ms/step - loss: 0.0068 - accuracy: 0.9979 - val_loss: 0.3654 - val_accuracy: 0.8654
Epoch 2/5
54/54 [==============================] - 10s 18

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 4ms/step - loss: 3.0288 - accuracy: 0.5879


In [84]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.09625285118818283, 0.2919221818447113], [0.12287917733192444, 0.5071086883544922], [0.1648828089237213, 0.8752776384353638], [0.11964885145425797, 1.0198652744293213], [0.29787641763687134, 1.54673171043396], [0.4365721344947815, 1.7502784729003906], [1.156044602394104, 2.4753761291503906], [1.8358707427978516, 2.8040149211883545], [1.7879897356033325, 2.75445294380188], [2.0196282863616943, 3.028756856918335]]
Accuracy for iterations:  [[0.9879561066627502, 0.8903218507766724], [0.9862908720970154, 0.821423351764679], [0.9732244610786438, 0.7205005884170532], [0.9810906052589417, 0.6993569731712341], [0.8586943745613098, 0.6328925490379333], [0.7638478875160217, 0.6436823606491089], [0.5811081528663635, 0.590823233127594], [0.5790995955467224, 0.5841749906539917], [0.584635853767395, 0.597199022769928], [0.5804653763771057, 0.5879350304603577]]
F1 for iterations:  [[0.9879585996628235, 0.8880029011684544], [0.9862914412058444, 0.8121348664985634], [0.97319277

#### #Epochs = 5, #Iterations = 15

In [85]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 15
# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [86]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1913.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [87]:
global_model = build_model((24,1,1))

In [88]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1913.hdf5')

Epoch 1/5


54/54 [==============================] - 10s 194ms/step - loss: 0.2479 - accuracy: 0.9556 - val_loss: 1.6721 - val_accuracy: 0.6400
Epoch 2/5
54/54 [==============================] - 10s 192ms/step - loss: 0.0310 - accuracy: 0.9924 - val_loss: 0.7515 - val_accuracy: 0.6422
Epoch 3/5
54/54 [==============================] - 10s 190ms/step - loss: 0.0249 - accuracy: 0.9925 - val_loss: 0.5051 - val_accuracy: 0.6767
Epoch 4/5
54/54 [==============================] - 10s 192ms/step - loss: 0.0232 - accuracy: 0.9925 - val_loss: 0.5618 - val_accuracy: 0.6519
Epoch 5/5
54/54 [==============================] - 10s 193ms/step - loss: 0.0217 - accuracy: 0.9927 - val_loss: 0.5475 - val_accuracy: 0.6590
Epoch 1/5
54/54 [==============================] - 10s 193ms/step - loss: 0.0200 - accuracy: 0.9933 - val_loss: 0.3913 - val_accuracy: 0.7403
Epoch 2/5
54/54 [==============================] - 10s 191ms/step - loss: 0.0173 - accuracy: 0.9944 - val_loss: 0.3801 - val_accuracy: 0.7717
Epoch 3/5
54/54 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 0.4205 - accuracy: 0.8769
Epoch 1/5
54/54 [==============================] - 11s 196ms/step - loss: 0.0151 - accuracy: 0.9957 - val_loss: 0.3374 - val_accuracy: 0.8307
Epoch 2/5
54/54 [==============================] - 10s 194ms/step - loss: 0.0145 - accuracy: 0.9959 - val_loss: 0.4684 - val_accuracy: 0.7910
Epoch 3/5
54/54 [==============================] - 11s 197ms/step - loss: 0.0144 - accuracy: 0.9958 - val_loss: 0.4533 - val_accuracy: 0.8001
Epoch 4/5
54/54 [==============================] - 10s 194ms/step - loss: 0.0142 - accuracy: 0.9960 - val_loss: 0.4413 - val_accuracy: 0.8081
Epoch 5/5
54/54 [==============================] - 11s 198ms/step - loss: 0.0135 - accuracy: 0.9963 - val_loss: 0.4525 - val_accuracy: 0.8089
Epoch 1/5
54/54 [==============================] - 11s 204ms/step - loss: 0.0131 - accuracy: 0.9962 - val_loss: 0.4575 - val_accuracy: 0.8284
Epoch 2/5
54/54 [==============================] - 11s 20

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


...

1721/1721 [==============================] - 8s 5ms/step - loss: 3.3669 - accuracy: 0.5978
Epoch 1/5
54/54 [==============================] - 12s 218ms/step - loss: 0.0067 - accuracy: 0.9983 - val_loss: 0.7341 - val_accuracy: 0.7993
Epoch 2/5
54/54 [==============================] - 12s 219ms/step - loss: 0.0059 - accuracy: 0.9983 - val_loss: 0.6709 - val_accuracy: 0.8084
Epoch 3/5
54/54 [==============================] - 12s 219ms/step - loss: 0.0058 - accuracy: 0.9984 - val_loss: 0.5303 - val_accuracy: 0.8337
Epoch 4/5
54/54 [==============================] - 12s 218ms/step - loss: 0.0063 - accuracy: 0.9982 - val_loss: 0.6994 - val_accuracy: 0.7977
Epoch 5/5
54/54 [==============================] - 12s 217ms/step - loss: 0.0058 - accuracy: 0.9984 - val_loss: 0.7557 - val_accuracy: 0.7952
Epoch 1/5
54/54 [==============================] - 12s 220ms/step - loss: 0.0064 - accuracy: 0.9981 - val_loss: 0.5858 - val_accuracy: 0.8311
Epoch 2/5
54/54 [==============================] - 12s 21

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 11s 6ms/step - loss: 3.4309 - accuracy: 0.6075
Epoch 1/5
54/54 [==============================] - 12s 222ms/step - loss: 0.0061 - accuracy: 0.9982 - val_loss: 0.4842 - val_accuracy: 0.8387
Epoch 2/5
54/54 [==============================] - 12s 222ms/step - loss: 0.0060 - accuracy: 0.9983 - val_loss: 0.7482 - val_accuracy: 0.7938
Epoch 3/5
54/54 [==============================] - 12s 220ms/step - loss: 0.0059 - accuracy: 0.9983 - val_loss: 0.7849 - val_accuracy: 0.7959
Epoch 4/5
54/54 [==============================] - 12s 220ms/step - loss: 0.0058 - accuracy: 0.9984 - val_loss: 0.6058 - val_accuracy: 0.8218
Epoch 5/5
54/54 [==============================] - 12s 229ms/step - loss: 0.0058 - accuracy: 0.9984 - val_loss: 0.5761 - val_accuracy: 0.8293
Epoch 1/5
54/54 [==============================] - 12s 222ms/step - loss: 0.0062 - accuracy: 0.9980 - val_loss: 0.6120 - val_accuracy: 0.8259
Epoch 2/5
54/54 [==============================] - 12s 2

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 3.5532 - accuracy: 0.6056


In [89]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.19311249256134033, 0.42053696513175964], [0.207519069314003, 0.7487212419509888], [0.2153296023607254, 1.1773408651351929], [0.20308400690555573, 1.4116877317428589], [0.39093777537345886, 1.8519067764282227], [0.584341287612915, 2.159454822540283], [0.8782985806465149, 2.4470252990722656], [1.0924036502838135, 2.6914265155792236], [1.4177628755569458, 2.7790586948394775], [1.517730474472046, 2.782214641571045], [1.855697512626648, 3.0099053382873535], [2.0155065059661865, 2.9566304683685303], [2.730656385421753, 3.3668792247772217], [2.6420443058013916, 3.4309136867523193], [2.999624252319336, 3.5531585216522217]]
Accuracy for iterations:  [[0.9832963347434998, 0.8768618702888489], [0.9705074429512024, 0.7520707845687866], [0.9358657598495483, 0.6496766805648804], [0.9405109882354736, 0.6331831812858582], [0.7832539677619934, 0.6055547595024109], [0.6764585375785828, 0.5994696021080017], [0.5852347612380981, 0.5861367583274841], [0.5860089659690857, 0.5898967

#### #Epochs = 10, #Iterations = 5

In [90]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5
# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 10

In [91]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1914.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [92]:
global_model = build_model((24,1,1))

In [93]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1914.hdf5')

Epoch 1/10


54/54 [==============================] - 12s 228ms/step - loss: 0.2533 - accuracy: 0.9727 - val_loss: 1.8887 - val_accuracy: 0.6399
Epoch 2/10
54/54 [==============================] - 12s 228ms/step - loss: 0.0327 - accuracy: 0.9924 - val_loss: 0.6309 - val_accuracy: 0.6492
Epoch 3/10
54/54 [==============================] - 12s 225ms/step - loss: 0.0257 - accuracy: 0.9926 - val_loss: 0.6105 - val_accuracy: 0.6527
Epoch 4/10
54/54 [==============================] - 12s 225ms/step - loss: 0.0239 - accuracy: 0.9926 - val_loss: 0.4485 - val_accuracy: 0.6978
Epoch 5/10
54/54 [==============================] - 12s 225ms/step - loss: 0.0226 - accuracy: 0.9928 - val_loss: 0.5086 - val_accuracy: 0.6659
Epoch 6/10
54/54 [==============================] - 12s 223ms/step - loss: 0.0208 - accuracy: 0.9932 - val_loss: 0.5316 - val_accuracy: 0.6664
Epoch 7/10
54/54 [==============================] - 12s 226ms/step - loss: 0.0193 - accuracy: 0.9939 - val_loss: 0.4171 - val_accuracy: 0.7278
Epoch 8/10

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 0.4283 - accuracy: 0.8469
Epoch 1/10
54/54 [==============================] - 12s 229ms/step - loss: 0.0140 - accuracy: 0.9961 - val_loss: 0.4353 - val_accuracy: 0.8186
Epoch 2/10
54/54 [==============================] - 12s 228ms/step - loss: 0.0132 - accuracy: 0.9962 - val_loss: 0.4390 - val_accuracy: 0.8173
Epoch 3/10
54/54 [==============================] - 12s 227ms/step - loss: 0.0134 - accuracy: 0.9963 - val_loss: 0.5241 - val_accuracy: 0.8004
Epoch 4/10
54/54 [==============================] - 12s 228ms/step - loss: 0.0129 - accuracy: 0.9965 - val_loss: 0.6577 - val_accuracy: 0.7691
Epoch 5/10
54/54 [==============================] - 12s 227ms/step - loss: 0.0129 - accuracy: 0.9965 - val_loss: 0.6586 - val_accuracy: 0.7787
Epoch 6/10
54/54 [==============================] - 12s 226ms/step - loss: 0.0128 - accuracy: 0.9964 - val_loss: 0.4950 - val_accuracy: 0.8198
Epoch 7/10
54/54 [==============================] -

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


...

1721/1721 [==============================] - 8s 5ms/step - loss: 1.8330 - accuracy: 0.5882: 4s - loss: 4.2361 - accuracy - ETA - ETA: 3s - loss: 3.2569 - accura
Epoch 1/10
54/54 [==============================] - 13s 232ms/step - loss: 0.0080 - accuracy: 0.9977 - val_loss: 0.4665 - val_accuracy: 0.8357
Epoch 2/10
54/54 [==============================] - 14s 260ms/step - loss: 0.0089 - accuracy: 0.9973 - val_loss: 1.0048 - val_accuracy: 0.7306
Epoch 3/10
54/54 [==============================] - 13s 246ms/step - loss: 0.0081 - accuracy: 0.9978 - val_loss: 0.6301 - val_accuracy: 0.8093
Epoch 4/10
54/54 [==============================] - 13s 237ms/step - loss: 0.0074 - accuracy: 0.9981 - val_loss: 0.6060 - val_accuracy: 0.8070
Epoch 5/10
54/54 [==============================] - 13s 240ms/step - loss: 0.0073 - accuracy: 0.9979 - val_loss: 0.5361 - val_accuracy: 0.8231
Epoch 6/10
54/54 [==============================] - 13s 232ms/step - loss: 0.0078 - accuracy: 0.9979 - val_loss: 0.5998 - va

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 25s 14ms/step - loss: 2.4086 - accuracy: 0.5780
Epoch 1/10
54/54 [==============================] - 15s 282ms/step - loss: 0.0069 - accuracy: 0.9982 - val_loss: 0.7027 - val_accuracy: 0.8013
Epoch 2/10
54/54 [==============================] - 15s 281ms/step - loss: 0.0066 - accuracy: 0.9983 - val_loss: 0.8368 - val_accuracy: 0.7796
Epoch 3/10
54/54 [==============================] - 15s 279ms/step - loss: 0.0070 - accuracy: 0.9982 - val_loss: 0.7806 - val_accuracy: 0.7800
Epoch 4/10
54/54 [==============================] - 15s 284ms/step - loss: 0.0065 - accuracy: 0.9982 - val_loss: 0.4830 - val_accuracy: 0.8351
Epoch 5/10
54/54 [==============================] - 15s 272ms/step - loss: 0.0065 - accuracy: 0.9982 - val_loss: 0.5774 - val_accuracy: 0.8127
Epoch 6/10
54/54 [==============================] - 16s 294ms/step - loss: 0.0065 - accuracy: 0.9981 - val_loss: 0.5144 - val_accuracy: 0.8337
Epoch 7/10
54/54 [==============================]

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 22s 13ms/step - loss: 2.6894 - accuracy: 0.5911


In [94]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.1365291327238083, 0.4283352196216583], [0.2651519179344177, 1.0024899244308472], [0.7037181258201599, 1.832952857017517], [1.5821514129638672, 2.4086146354675293], [2.3268370628356934, 2.6893749237060547]]
Accuracy for iterations:  [[0.9855824112892151, 0.8469265699386597], [0.860381543636322, 0.6725096106529236], [0.6239592432975769, 0.5882256627082825], [0.5767039656639099, 0.5780171751976013], [0.5847015976905823, 0.5910593867301941]]
F1 for iterations:  [[0.9855826670707051, 0.8409762834970507], [0.8568613387554346, 0.6149381434584613], [0.5508425246294328, 0.4636867477882422], [0.46738062441285394, 0.4423453790972255], [0.4837946867218398, 0.47302792034124125]]
Precision for iterations:  [[0.9855831374686987, 0.8727007844361674], [0.8850718746255002, 0.7805118410349414], [0.7680038756565108, 0.7443149963827191], [0.7453524292311242, 0.735034928827699], [0.7414682594253333, 0.7193566642547569]]
Recall for iterations:  [[0.9855824008881358, 0.84692654217830

#### #Epochs = 8, #Iterations = 5

In [95]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5
# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 8

In [96]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1915.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [97]:
global_model = build_model((24,1,1))

In [98]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1915.hdf5')

Epoch 1/8


54/54 [==============================] - 12s 228ms/step - loss: 0.2470 - accuracy: 0.9317 - val_loss: 1.6157 - val_accuracy: 0.6400
Epoch 2/8
54/54 [==============================] - 12s 222ms/step - loss: 0.0311 - accuracy: 0.9925 - val_loss: 0.7175 - val_accuracy: 0.6467
Epoch 3/8
54/54 [==============================] - 12s 219ms/step - loss: 0.0264 - accuracy: 0.9925 - val_loss: 0.6154 - val_accuracy: 0.6488
Epoch 4/8
54/54 [==============================] - 12s 217ms/step - loss: 0.0246 - accuracy: 0.9926 - val_loss: 0.5837 - val_accuracy: 0.6496
Epoch 5/8
54/54 [==============================] - 12s 217ms/step - loss: 0.0230 - accuracy: 0.9926 - val_loss: 0.4128 - val_accuracy: 0.7236
Epoch 6/8
54/54 [==============================] - 12s 217ms/step - loss: 0.0211 - accuracy: 0.9933 - val_loss: 0.4062 - val_accuracy: 0.7333
Epoch 7/8
54/54 [==============================] - 12s 216ms/step - loss: 0.0192 - accuracy: 0.9939 - val_loss: 0.4178 - val_accuracy: 0.7333
Epoch 8/8
54/54 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 17s 10ms/step - loss: 0.3090 - accuracy: 0.9097
Epoch 1/8
54/54 [==============================] - 13s 232ms/step - loss: 0.0148 - accuracy: 0.9958 - val_loss: 0.5605 - val_accuracy: 0.7666
Epoch 2/8
54/54 [==============================] - 12s 217ms/step - loss: 0.0140 - accuracy: 0.9960 - val_loss: 0.5251 - val_accuracy: 0.7822
Epoch 3/8
54/54 [==============================] - 12s 218ms/step - loss: 0.0142 - accuracy: 0.9959 - val_loss: 0.3481 - val_accuracy: 0.8477
Epoch 4/8
54/54 [==============================] - 12s 217ms/step - loss: 0.0137 - accuracy: 0.9961 - val_loss: 0.6239 - val_accuracy: 0.7772
Epoch 5/8
54/54 [==============================] - 12s 215ms/step - loss: 0.0136 - accuracy: 0.9962 - val_loss: 0.4505 - val_accuracy: 0.8222
Epoch 6/8
54/54 [==============================] - 12s 217ms/step - loss: 0.0132 - accuracy: 0.9962 - val_loss: 0.4862 - val_accuracy: 0.8098
Epoch 7/8
54/54 [==============================] - 12s 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


...

1721/1721 [==============================] - 16s 9ms/step - loss: 1.1401 - accuracy: 0.6861
Epoch 1/8
54/54 [==============================] - 12s 214ms/step - loss: 0.0084 - accuracy: 0.9978 - val_loss: 0.6121 - val_accuracy: 0.8078
Epoch 2/8
54/54 [==============================] - 12s 215ms/step - loss: 0.0085 - accuracy: 0.9977 - val_loss: 0.9500 - val_accuracy: 0.7377
Epoch 3/8
54/54 [==============================] - 12s 220ms/step - loss: 0.0079 - accuracy: 0.9978 - val_loss: 0.4002 - val_accuracy: 0.8463
Epoch 4/8
54/54 [==============================] - 13s 232ms/step - loss: 0.0077 - accuracy: 0.9978 - val_loss: 0.7166 - val_accuracy: 0.7872
Epoch 5/8
54/54 [==============================] - 12s 217ms/step - loss: 0.0079 - accuracy: 0.9979 - val_loss: 0.6051 - val_accuracy: 0.8065
Epoch 6/8
54/54 [==============================] - 11s 203ms/step - loss: 0.0080 - accuracy: 0.9979 - val_loss: 0.5437 - val_accuracy: 0.8161
Epoch 7/8
54/54 [==============================] - 11s 2

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 6s 4ms/step - loss: 1.7186 - accuracy: 0.6793
Epoch 1/8
54/54 [==============================] - 11s 196ms/step - loss: 0.0072 - accuracy: 0.9981 - val_loss: 0.6565 - val_accuracy: 0.7961
Epoch 2/8
54/54 [==============================] - 11s 196ms/step - loss: 0.0071 - accuracy: 0.9981 - val_loss: 0.5742 - val_accuracy: 0.8095
Epoch 3/8
54/54 [==============================] - 11s 196ms/step - loss: 0.0067 - accuracy: 0.9982 - val_loss: 0.6854 - val_accuracy: 0.7932
Epoch 4/8
54/54 [==============================] - 10s 194ms/step - loss: 0.0067 - accuracy: 0.9980 - val_loss: 0.5630 - val_accuracy: 0.8158
Epoch 5/8
54/54 [==============================] - 11s 196ms/step - loss: 0.0070 - accuracy: 0.9982 - val_loss: 0.5873 - val_accuracy: 0.8081
Epoch 6/8
54/54 [==============================] - 11s 196ms/step - loss: 0.0067 - accuracy: 0.9982 - val_loss: 0.7150 - val_accuracy: 0.7843
Epoch 7/8
54/54 [==============================] - 10s 19

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 8s 5ms/step - loss: 2.1888 - accuracy: 0.6872


In [99]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.1551613062620163, 0.3090450167655945], [0.39441975951194763, 0.827255368232727], [0.48933789134025574, 1.1401209831237793], [1.3998279571533203, 1.718559980392456], [2.3852107524871826, 2.1888344287872314]]
Accuracy for iterations:  [[0.9876712560653687, 0.9097035527229309], [0.7235531210899353, 0.7248964905738831], [0.6320590972900391, 0.686133086681366], [0.5892736911773682, 0.6792668700218201], [0.5906614065170288, 0.6871684789657593]]
F1 for iterations:  [[0.9876740047741682, 0.9084097026691861], [0.6970280822167986, 0.6915926298900763], [0.5656337469397139, 0.6355295358394175], [0.4935263554174655, 0.6249145617594916], [0.49617727724775007, 0.6368594826811318]]
Precision for iterations:  [[0.987710573856323, 0.917019485897581], [0.8072991844827647, 0.8046626074435853], [0.7640255800302556, 0.788097391066705], [0.7370750903214642, 0.7857416156544874], [0.7372821739744763, 0.7895831169954107]]
Recall for iterations:  [[0.9876712728972509, 0.9097035530044322

#### #Epochs = 15, #Iterations = 5

In [100]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5
# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [101]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1916.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [102]:
global_model = build_model((24,1,1))

In [103]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1916.hdf5')

Epoch 1/15


54/54 [==============================] - 11s 207ms/step - loss: 0.2578 - accuracy: 0.9609 - val_loss: 1.8913 - val_accuracy: 0.6399
Epoch 2/15
54/54 [==============================] - 11s 195ms/step - loss: 0.0326 - accuracy: 0.9923 - val_loss: 0.6685 - val_accuracy: 0.6476
Epoch 3/15
54/54 [==============================] - 11s 201ms/step - loss: 0.0257 - accuracy: 0.9925 - val_loss: 0.5236 - val_accuracy: 0.6765
Epoch 4/15
54/54 [==============================] - 11s 211ms/step - loss: 0.0237 - accuracy: 0.9925 - val_loss: 0.4728 - val_accuracy: 0.6862
Epoch 5/15
54/54 [==============================] - 11s 209ms/step - loss: 0.0219 - accuracy: 0.9930 - val_loss: 0.4140 - val_accuracy: 0.7261
Epoch 6/15
54/54 [==============================] - 11s 206ms/step - loss: 0.0196 - accuracy: 0.9938 - val_loss: 0.4572 - val_accuracy: 0.7128
Epoch 7/15
54/54 [==============================] - 11s 210ms/step - loss: 0.0179 - accuracy: 0.9945 - val_loss: 0.3424 - val_accuracy: 0.7901
Epoch 8/15

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 0.6025 - accuracy: 0.8243
Epoch 1/15
54/54 [==============================] - 12s 218ms/step - loss: 0.0129 - accuracy: 0.9967 - val_loss: 0.5113 - val_accuracy: 0.8050
Epoch 2/15
54/54 [==============================] - 11s 210ms/step - loss: 0.0122 - accuracy: 0.9967 - val_loss: 0.4126 - val_accuracy: 0.8418
Epoch 3/15
54/54 [==============================] - 11s 208ms/step - loss: 0.0123 - accuracy: 0.9967 - val_loss: 0.4079 - val_accuracy: 0.8478
Epoch 4/15
54/54 [==============================] - 11s 212ms/step - loss: 0.0120 - accuracy: 0.9968 - val_loss: 0.4019 - val_accuracy: 0.8435
Epoch 5/15
54/54 [==============================] - 11s 210ms/step - loss: 0.0110 - accuracy: 0.9969 - val_loss: 0.5778 - val_accuracy: 0.7965
Epoch 6/15
54/54 [==============================] - 11s 212ms/step - loss: 0.0110 - accuracy: 0.9970 - val_loss: 0.4398 - val_accuracy: 0.8437
Epoch 7/15
54/54 [==============================] -

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


...

1721/1721 [==============================] - 9s 5ms/step - loss: 2.0876 - accuracy: 0.6579
Epoch 1/15
54/54 [==============================] - 11s 205ms/step - loss: 0.0069 - accuracy: 0.9981 - val_loss: 0.4917 - val_accuracy: 0.8369
Epoch 2/15
54/54 [==============================] - 11s 201ms/step - loss: 0.0068 - accuracy: 0.9982 - val_loss: 0.5841 - val_accuracy: 0.8159
Epoch 3/15
54/54 [==============================] - 11s 197ms/step - loss: 0.0065 - accuracy: 0.9982 - val_loss: 0.7025 - val_accuracy: 0.8043
Epoch 4/15
54/54 [==============================] - 10s 194ms/step - loss: 0.0064 - accuracy: 0.9983 - val_loss: 0.6358 - val_accuracy: 0.8102
Epoch 5/15
54/54 [==============================] - 11s 195ms/step - loss: 0.0066 - accuracy: 0.9982 - val_loss: 0.8439 - val_accuracy: 0.7726
Epoch 6/15
54/54 [==============================] - 11s 196ms/step - loss: 0.0064 - accuracy: 0.9982 - val_loss: 0.9817 - val_accuracy: 0.7390
Epoch 7/15
54/54 [==============================] -

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 10s 6ms/step - loss: 2.1810 - accuracy: 0.7010
Epoch 1/15
54/54 [==============================] - 11s 204ms/step - loss: 0.0064 - accuracy: 0.9982 - val_loss: 0.4371 - val_accuracy: 0.8478
Epoch 2/15
54/54 [==============================] - 11s 200ms/step - loss: 0.0062 - accuracy: 0.9984 - val_loss: 0.5495 - val_accuracy: 0.8274
Epoch 3/15
54/54 [==============================] - 11s 196ms/step - loss: 0.0058 - accuracy: 0.9983 - val_loss: 0.6802 - val_accuracy: 0.8021
Epoch 4/15
54/54 [==============================] - 11s 195ms/step - loss: 0.0061 - accuracy: 0.9984 - val_loss: 0.5591 - val_accuracy: 0.8186
Epoch 5/15
54/54 [==============================] - 10s 193ms/step - loss: 0.0055 - accuracy: 0.9985 - val_loss: 0.6556 - val_accuracy: 0.8105
Epoch 6/15
54/54 [==============================] - 11s 196ms/step - loss: 0.0078 - accuracy: 0.9982 - val_loss: 0.7606 - val_accuracy: 0.7853
Epoch 7/15
54/54 [==============================] 

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 2.4714 - accuracy: 0.7287


In [104]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.21554751694202423, 0.6025245785713196], [0.43903639912605286, 1.4645129442214966], [1.2050604820251465, 2.087563991546631], [1.522641658782959, 2.1810131072998047], [2.0096006393432617, 2.4713964462280273]]
Accuracy for iterations:  [[0.951904833316803, 0.8243297338485718], [0.7666671276092529, 0.6635181307792664], [0.5929402112960815, 0.6579052805900574], [0.59264075756073, 0.7010281085968018], [0.5913698673248291, 0.7287473678588867]]
F1 for iterations:  [[0.9517577984522455, 0.815554872472297], [0.7510943322934576, 0.6004197509719447], [0.5016871463562266, 0.5910450327209422], [0.5031850823105745, 0.6578276036984575], [0.5035784985272762, 0.6969376973614473]]
Precision for iterations:  [[0.9537582253850795, 0.8581842500087389], [0.8281696470343505, 0.7777392125236178], [0.7319174911136448, 0.7766280385398969], [0.7224876012655619, 0.7937685972410671], [0.7104288177094473, 0.8061525739118192]]
Recall for iterations:  [[0.9519048175523679, 0.8243297246239919]

#### #Epochs = 5, #Iterations = 3

In [153]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_17376/1685572531.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low

In [154]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3
# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [155]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id1917.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [156]:
global_model = build_model((24,1,1))

In [157]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    w_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        w_list.append(local_model.get_weights())

    avg_w = aggregate(w_list)
    global_model.set_weights(avg_w) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) # Evaluate with test basic to know progress
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus) # Evaluate with test plus to know progress

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id1917.hdf5')

Epoch 1/5
45/45 [==============================] - 10s 213ms/step - loss: 0.2985 - accuracy: 0.9581 - val_loss: 2.1682 - val_accuracy: 0.5722
Epoch 2/5
45/45 [==============================] - 9s 210ms/step - loss: 0.0375 - accuracy: 0.9908 - val_loss: 0.6280 - val_accuracy: 0.6188
Epoch 3/5
45/45 [==============================] - 9s 209ms/step - loss: 0.0259 - accuracy: 0.9911 - val_loss: 0.5202 - val_accuracy: 0.6711
Epoch 4/5
45/45 [==============================] - 9s 207ms/step - loss: 0.0238 - accuracy: 0.9914 - val_loss: 0.4142 - val_accuracy: 0.7493
Epoch 5/5
45/45 [==============================] - 9s 208ms/step - loss: 0.0219 - accuracy: 0.9921 - val_loss: 0.3805 - val_accuracy: 0.7698
Epoch 1/5
65/65 [==============================] - 14s 209ms/step - loss: 0.0209 - accuracy: 0.9936 - val_loss: 0.4311 - val_accuracy: 0.7335
Epoch 2/5
65/65 [==============================] - 13s 207ms/step - loss: 0.0176 - accuracy: 0.9946 - val_loss: 0.4078 - val_accuracy: 0.7782
Epoch 3/5


c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 0.2117 - accuracy: 0.9045
Epoch 1/5
45/45 [==============================] - 9s 206ms/step - loss: 0.0155 - accuracy: 0.9950 - val_loss: 0.4115 - val_accuracy: 0.7848
Epoch 2/5
45/45 [==============================] - 10s 212ms/step - loss: 0.0147 - accuracy: 0.9952 - val_loss: 0.3895 - val_accuracy: 0.8045
Epoch 3/5
45/45 [==============================] - 9s 207ms/step - loss: 0.0145 - accuracy: 0.9956 - val_loss: 0.2868 - val_accuracy: 0.8383
Epoch 4/5
45/45 [==============================] - 9s 204ms/step - loss: 0.0140 - accuracy: 0.9958 - val_loss: 0.4138 - val_accuracy: 0.7975
Epoch 5/5
45/45 [==============================] - 9s 207ms/step - loss: 0.0140 - accuracy: 0.9958 - val_loss: 0.3500 - val_accuracy: 0.8234
Epoch 1/5
65/65 [==============================] - 13s 206ms/step - loss: 0.0145 - accuracy: 0.9960 - val_loss: 0.3666 - val_accuracy: 0.8444
Epoch 2/5
65/65 [==============================] - 13s 204ms/

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 9s 5ms/step - loss: 0.4155 - accuracy: 0.8119
Epoch 1/5
45/45 [==============================] - 9s 208ms/step - loss: 0.0136 - accuracy: 0.9957 - val_loss: 0.4478 - val_accuracy: 0.8064
Epoch 2/5
45/45 [==============================] - 9s 208ms/step - loss: 0.0131 - accuracy: 0.9959 - val_loss: 0.3384 - val_accuracy: 0.8421
Epoch 3/5
45/45 [==============================] - 9s 207ms/step - loss: 0.0128 - accuracy: 0.9960 - val_loss: 0.4207 - val_accuracy: 0.8158
Epoch 4/5
45/45 [==============================] - 9s 203ms/step - loss: 0.0128 - accuracy: 0.9960 - val_loss: 0.3400 - val_accuracy: 0.8356
Epoch 5/5
45/45 [==============================] - 9s 207ms/step - loss: 0.0127 - accuracy: 0.9961 - val_loss: 0.4374 - val_accuracy: 0.8109
Epoch 1/5
65/65 [==============================] - 13s 206ms/step - loss: 0.0125 - accuracy: 0.9965 - val_loss: 0.5329 - val_accuracy: 0.8134
Epoch 2/5
65/65 [==============================] - 13s 204ms/s

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


1721/1721 [==============================] - 7s 4ms/step - loss: 0.8198 - accuracy: 0.6863


In [158]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.055341463536024094, 0.21167635917663574], [0.054742418229579926, 0.4154618978500366], [0.09591921418905258, 0.8197669386863708]]
Accuracy for iterations:  [[0.984479546546936, 0.9045266509056091], [0.9725160002708435, 0.8118687868118286], [0.9642627835273743, 0.6862965822219849]]
F1 for iterations:  [[0.9844785269044698, 0.9029244478672587], [0.9724731963312149, 0.8011067238666677], [0.9641574953615748, 0.6362723102708954]]
Precision for iterations:  [[0.9844799221101318, 0.913737606198411], [0.9732125269378484, 0.8510417498580631], [0.9661124994480643, 0.7859974529417739]]
Recall for iterations:  [[0.9844795348973093, 0.904526629368597], [0.9725159952087411, 0.8118687786093148], [0.9642627596482515, 0.6862965923127226]]
